This script is used to download the data from the FMI open database.

In [3]:
import datetime as dt
from fmiopendata.wfs import download_stored_query
import math
import json

In [4]:
# Retrieve data for year 2019
# NOTE: This is slow as fuck!

all_obs = []
abs_start_time = dt.datetime(2019, 1, 1)

for i in range(365):
    # Retrieve the latest week of data from a bounding box
    start_time = abs_start_time + dt.timedelta(days=i)
    end_time = abs_start_time + dt.timedelta(days=i+1)
    
    # Convert times to properly formatted strings
    start_time = start_time.isoformat(timespec="seconds") + "Z" # -> 2020-07-07T12:00:00Z
    end_time = end_time.isoformat(timespec="seconds") + "Z"

    # API call
    obs = download_stored_query("fmi::observations::weather::multipointcoverage",
                                args=["bbox=19,59,32,70",
                                      "starttime=" + start_time,
                                      "endtime=" + end_time,
                                      "timeseries=True"])
    all_obs.append(obs)
    print(i, flush=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
# Check number of days
print(len(all_obs))

365


In [60]:
# Preprosess data

# Definitions
places = ['Alajärvi Möksy', 'Asikkala Pulkkilanharju', 'Enontekiö Kilpisjärvi Saana', 'Enontekiö Kilpisjärvi kyläkeskus', 'Enontekiö Näkkälä', 'Espoo Nuuksio', 'Espoo Tapiola', 'Haapavesi Mustikkamäki', 'Hailuoto Keskikylä', 'Hailuoto Marjaniemi', 'Halsua Purola', 'Hammarland Märket', 'Hanko Russarö', 'Hanko Tulliniemi', 'Hanko Tvärminne', 'Hattula Lepaa', 'Heinola Asemantaus', 'Helsinki Harmaja', 'Helsinki Kaisaniemi', 'Helsinki Kumpula', 'Helsinki Malmi lentokenttä', 'Helsinki Vuosaari Käärmeniementie', 'Helsinki Vuosaari satama', 'Hyvinkää Hyvinkäänkylä', 'Hämeenlinna Lammi Evo', 'Hämeenlinna Lammi Pappila', 'Ilomantsi Mekrijärvi', 'Inari Angeli Lintupuoliselkä', 'Inari Ivalo lentoasema', 'Inari Kaamanen', 'Inari Kirakkajärvi', 'Inari Nellim', 'Inari Raja-Jooseppi', 'Inari Saariselkä Kaunispää', 'Inari Saariselkä matkailukeskus', 'Inari Seitalaassa', 'Inari Väylä', 'Inkoo Bågaskär', 'Joensuu Linnunlahti', 'Jokioinen Ilmala', 'Jomala Jomalaby', 'Jomala Maarianhamina lentoasema', 'Joutsa Savenaho', 'Juuka Niemelä', 'Juupajoki Hyytiälä', 'Juva Partala', 'Jyväskylä lentoasema', 'Jämsä Halli Lentoasemantie', 'Jämsä Halli lentoasema', 'Järvenpää Sorto', 'Kaarina Yltöinen', 'Kajaani Petäisenniska', 'Kajaani lentoasema', 'Kalajoki Ulkokalla', 'Kankaanpää Niinisalo lentokenttä', 'Karvia Alkkia', 'Kaskinen Sälgrund', 'Kauhajoki Kuja-Kokko', 'Kauhava lentokenttä', 'Kemi Ajos', 'Kemi I majakka', 'Kemi Kemi-Tornio lentoasema', 'Kemijärvi lentokenttä', 'Kemiönsaari Kemiö', 'Kemiönsaari Vänö', 'Kirkkonummi Mäkiluoto', 'Kittilä Kenttärova', 'Kittilä Lompolonvuoma', 'Kittilä Matorova', 'Kittilä Pokka', 'Kittilä kirkonkylä', 'Kittilä lentoasema', 'Kokemäki Tulkkila', 'Kokkola Tankar', 'Korsnäs Bredskäret', 'Kotka Haapasaari', 'Kotka Rankki', 'Kouvola Anjala', 'Kouvola Utti Lentoportintie', 'Kouvola Utti lentoasema', 'Kristiinankaupunki Majakka', 'Kruunupyy Kokkola-Pietarsaari lentoasema', 'Kuhmo Kalliojoki', 'Kumlinge kirkonkylä', 'Kuopio Maaninka', 'Kuopio Ritoniemi', 'Kuopio Savilahti', 'Kustavi Isokari', 'Kuusamo Juuma', 'Kuusamo Kiutaköngäs', 'Kuusamo Ruka Talvijärvi', 'Kuusamo Rukatunturi', 'Kuusamo lentoasema', 'Kökar Bogskär', 'Lahti Sopenkorpi', 'Lappeenranta Hiekkapakka', 'Lappeenranta Konnunsuo', 'Lappeenranta Lepola', 'Lappeenranta lentoasema', 'Lemland Nyhamn', 'Lieksa Lampela', 'Liperi Joensuu lentoasema', 'Liperi Tuiskavanluoto', 'Lohja Porla', 'Loviisa Orrengrund', 'Luhanka Judinsalo', 'Lumparland Långnäs satama', 'Maalahti Strömmingsbådan', 'Maarianhamina Länsisatama', 'Mikkeli lentoasema', 'Multia Karhila', 'Muonio Laukukero', 'Muonio Sammaltunturi', 'Muonio kirkonkylä', 'Mustasaari Valassaaret', 'Mäntsälä Hirvihaara', 'Nurmes Valtimo', 'Nurmijärvi Röykkä', 'Oulu Oulunsalo Pellonpää', 'Oulu Vihreäsaari satama', 'Oulu lentoasema', 'Parainen Fagerholm', 'Parainen Utö', 'Parikkala Koitsanlahti', 'Pelkosenniemi Pyhätunturi', 'Pello kirkonkylä', 'Pietarsaari Kallan', 'Pirkkala Tampere-Pirkkala lentoasema', 'Pori Tahkoluoto satama', 'Pori lentoasema', 'Pori rautatieasema', 'Porvoo Emäsalo', 'Porvoo Harabacka', 'Porvoo Kilpilahti satama', 'Pudasjärvi lentokenttä', 'Puolanka Paljakka', 'Puumala kirkonkylä', 'Pyhäjärvi Ojakylä', 'Raahe Lapaluoto satama', 'Raasepori Jussarö', 'Rantasalmi Rukkasluoto', 'Ranua lentokenttä', 'Rauma Kylmäpihlaja', 'Rauma Pyynpää', 'Rautavaara Ylä-Luosta', 'Rovaniemi Apukka', 'Rovaniemi lentoasema', 'Rovaniemi rautatieasema', 'Salla Naruska', 'Salla Värriötunturi', 'Salla kirkonkylä', 'Salo Kiikala lentokenttä', 'Salo Kärkkä', 'Savonlinna Punkaharju Laukansaari', 'Savonlinna lentoasema', 'Savukoski kirkonkylä', 'Seinäjoki Pelmaa', 'Siikajoki Ruukki', 'Siilinjärvi Kuopio lentoasema', 'Sipoo Eestiluoto', 'Sipoo Itätoukki', 'Sodankylä Lokka', 'Sodankylä Tähtelä', 'Sodankylä Vuotso', 'Somero Salkola', 'Sotkamo Kuolaniemi', 'Suomussalmi Pesiö', 'Taivalkoski kirkonkylä', 'Tampere Härmälä', 'Tampere Siilinkari', 'Tampere Tampella', 'Tohmajärvi Kemie', 'Toholampi Laitala', 'Tornio Torppi', 'Turku Artukainen', 'Turku Rajakari', 'Turku lentoasema', 'Utsjoki Kevo', 'Utsjoki Kevo Kevojärvi', 'Vaala Pelso', 'Vaasa Klemettilä', 'Vaasa lentoasema', 'Vantaa Helsinki-Vantaan lentoasema', 'Varkaus Kosulanniemi', 'Vesanto kirkonkylä', 'Vieremä Kaarakkala', 'Vihti Maasoja', 'Viitasaari Haapaniemi', 'Virolahti Koivuniemi', 'Virrat Äijänneva', 'Ylitornio Meltosjärvi', 'Ylivieska lentokenttä', 'Ähtäri Inha']
# features = ['t2m', 'ws_10min', 'wd_10min', 'rh','p_sea']
features = ['t2m', 'ws_10min', 'wd_10min', 'rh','p_sea']

# Shape
data = {f : {} for f in features}
for f in features:
    data[f] = {p : [] for p in places}
    
# Places to remove if lacking features or data alltogether 
remove = []

# Change data format
for i in range(len(all_obs[:31])):
    print(i)
    for p in places:
        # Check that place exists in data
        if p in all_obs[i].data.keys():
            for f in features:
                place = all_obs[i].data[p]
                values = place[f]["values"]
                data[f][p] += [[place["times"][i].isoformat(timespec="seconds"), values[i]] for i in range(0, len(values), 6)]
        else:
            remove.append(p)
        
# Remove places that lack features
for f in features:
    for p in places:
        if math.isnan(data[f][p][0][1]):
            remove.append(p)

for f in features:
    for r in remove:
        data[f].pop(r, None)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [67]:
for i in range(len(all_obs)):
    print(i)
    with open("./raw/data{}.json".format(i), "w", encoding="utf8") as outfile:
        # Shape
        data = {f : {} for f in features}
        for f in features:
            data[f] = {p : [] for p in places}
            
        for p in places:
            # Check that place exists in data
            if p in all_obs[i].data.keys():
                for f in features:
                    place = all_obs[i].data[p]
                    values = place[f]["values"]
                    data[f][p] = [[place["times"][i].isoformat(timespec="seconds"), values[i]] for i in range(0, len(values), 6)]
        json.dump(data, outfile, ensure_ascii=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [56]:
s = 0
for p in places:
    if not math.isnan(all_obs[0].data[p]["p_sea"]["values"][0]):
        s += 1
print(s)

142


In [34]:
'Porvoo Kilpilahti satama' in all_obs[1].data

True

In [36]:
# Check number of observations
data["t2m"].keys()

dict_keys([])

In [11]:
# Write the data
with open('data.json', 'w', encoding="utf8") as outfile:
    json.dump(data, outfile, ensure_ascii=False)